In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/layoutlm/medquad.csv
/kaggle/input/layoutlm/models--yikuan8--Clinical-Longformer/refs/main
/kaggle/input/layoutlm/models--yikuan8--Clinical-Longformer/blobs/75801df688e89a96f642b98cbdd97288f5207518
/kaggle/input/layoutlm/models--yikuan8--Clinical-Longformer/blobs/dcbb04799764dccba6c5dfeef25f62e9955b7d07
/kaggle/input/layoutlm/models--yikuan8--Clinical-Longformer/blobs/87e239f555459c3668993fbf5b53bea93f503be86f2f4285f431fdb2c6c33c75
/kaggle/input/layoutlm/models--yikuan8--Clinical-Longformer/blobs/6636bda4a1fd7a63653dffb22683b8162c8de956
/kaggle/input/layoutlm/models--yikuan8--Clinical-Longformer/blobs/2ea7ad0e45a9d1d1591782ba7e29a703d0758831
/kaggle/input/layoutlm/models--yikuan8--Clinical-Longformer/blobs/49684d2d368a3a66b7056f79ebeb7d236a685f6d
/kaggle/input/layoutlm/models--yikuan8--Clinical-Longformer/blobs/4ebe4bb3f3114daf2e4cc349f24873a1175a35d7
/kaggle/input/layoutlm/models--yikuan8--Clinical-Longformer/.no_exist/dc05ee5437027609b953618bc8e2b725a30bd670/added_token

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [3]:
# 🔧 Step 1: Install Dependencies
!pip install transformers datasets sentencepiece accelerate -q
!pip install peft bitsandbytes -q

In [4]:
# 🔗 Step 2: Load Required Libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer
import os

2025-06-06 13:41:33.583140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749217293.606819     231 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749217293.614170     231 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# 📌 Step 3: Load NLLB-200 for Translation
nllb_model_name = "facebook/nllb-200-distilled-600M"
translator_tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)
translator_model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model_name).to("cuda")

def translate(text, source_lang, target_lang):
    encoded = translator_tokenizer(text, return_tensors="pt").to("cuda")
    target_token = target_lang
    target_token_id = translator_tokenizer.convert_tokens_to_ids(target_token)
    outputs = translator_model.generate(
        **encoded,
        forced_bos_token_id=target_token_id,
        max_length=512
    )
    return translator_tokenizer.decode(outputs[0], skip_special_tokens=True)


In [6]:
# 🧠 Step 4: Load and Prepare TinyLLaMA for Fine-Tuning
tinyllama_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(tinyllama_model_name)
model = AutoModelForCausalLM.from_pretrained(tinyllama_model_name, quantization_config=bnb_config, device_map="auto")
model = prepare_model_for_kbit_training(model)

In [7]:
# ⚙️ Step 5: Apply LoRA for Efficient Fine-tuning
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [8]:
# 📄 Step 6: Load MedQuAD Dataset from Uploaded CSV
from datasets import Dataset
import pandas as pd

df = pd.read_csv("/kaggle/input/layoutlm/medquad.csv")  # Make sure this file is uploaded to Colab
# Convert to instruction-style fine-tuning format
def format_data(example):
    return {
        "instruction": f"Explain in simple terms: {example['question']}",
        "input": "",
        "output": example["answer"]
    }

dataset = Dataset.from_pandas(df[['question', 'answer']].dropna())
dataset = dataset.map(format_data)
dataset = dataset.train_test_split(test_size=0.1)

Map:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [9]:
def tokenize(example):
    prompt = f"<s>[INST] {example['instruction']} [/INST] {example['output']} </s>"
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized
tokenized_data = dataset.map(tokenize)


Map:   0%|          | 0/14766 [00:00<?, ? examples/s]

Map:   0%|          | 0/1641 [00:00<?, ? examples/s]

In [10]:
training_args = TrainingArguments(
    output_dir="./tinyllama-medquad",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    logging_steps=10,
    save_steps=100,
    eval_strategy="epoch",
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"]
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.433200,0.618322
2,0.541900,0.608509


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=7384, training_loss=0.6394273047037786, metrics={'train_runtime': 10261.428, 'train_samples_per_second': 2.878, 'train_steps_per_second': 0.72, 'total_flos': 9.399232485339955e+16, 'train_loss': 0.6394273047037786, 'epoch': 2.0})

In [11]:
# 💬 Step 10: Inference with Translation
def generate_response(prompt, max_new_tokens=256):
    full_prompt = f"<s>[INST] {prompt.strip()} [/INST]"
    input_ids = tokenizer(full_prompt, return_tensors="pt").input_ids.to(model.device)
    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9
    )
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    if "[/INST]" in response:
        response = response.split("[/INST]")[-1].strip()
    return response

In [12]:
# 🏥 Step 11: Rural Health Bot Function
def rural_health_bot(user_input_indic, src_lang="hin_Deva"):
    # Step 1: Translate Indic -> English
    user_input_en = translate(user_input_indic, source_lang=src_lang, target_lang="eng_Latn")
    # Step 2: Generate response in English
    response_en = generate_response(user_input_en)
    # Step 3: Translate back to Indic
    final_response = translate(response_en, source_lang="eng_Latn", target_lang=src_lang)
    return final_response

In [13]:
# 🧪 Step 12: Test
hindi_question = "मुझे एक हफ्ते से खांसी हो रही है, इसका क्या कारण हो सकता है?"
print("Answer:", rural_health_bot(hindi_question))


Answer: मैं एक सप्ताह से खासी कर रहा हूं, इसका कारण क्या हो सकता है? सर्दी के लक्षण श्वसन रोग के लक्षणों से काफी अलग हो सकते हैं। यदि आपको सर्दी है, तो आप यह सुनिश्चित नहीं हैं कि यह वायरस है या कुछ और, और आप बहुत खासी कर रहे हैं, तो आप शायद एक वायरस खासी कर रहे हैं। सर्दी के लक्षणों में गले में दर्द, नाक बह रहा है, खासी और एक दर्दनाक या दर्दनाक छाती शामिल है। यह भी आम है कि सर्दी के साथ बुखार और गर्दन में दर्द हो। सर्दी वायरस, बैक्टीरियल संक्रमण या एक कवक संक्रमण से हो सकती है। बैक्टीरियल संक्रमण सबसे आम कारण है सर्दी। वायरस सामान्य कवक का कारण है। वायरस भी सामान्य सर्दी का कारण है। फ्लू संक्रमण, जैसे कि कुछ खमीर संक्रमण, अक्सर एक सर्दी का कारण होता है। लोगों में सिरदर्द, सिरदर्द, सिरदर्द, सिरदर्द, सिंकल के लक्षण भी शामिल हैं।


In [15]:
# Save model
trainer.save_model("./tinyllama-medquad")

# Save tokenizer
tokenizer.save_pretrained("./tinyllama-medquad")


('./tinyllama-medquad/tokenizer_config.json',
 './tinyllama-medquad/special_tokens_map.json',
 './tinyllama-medquad/tokenizer.model',
 './tinyllama-medquad/added_tokens.json',
 './tinyllama-medquad/tokenizer.json')

In [16]:
import shutil

# Zip the folder
shutil.make_archive("tinyllama-medquad", 'zip', "./tinyllama-medquad")


'/kaggle/working/tinyllama-medquad.zip'

In [21]:
def rural_health_bot(user_input_indic, src_lang="tel_Telu"):
    # Step 1: Translate Indic -> English
    user_input_en = translate(user_input_indic, source_lang=src_lang, target_lang="eng_Latn")
    # Step 2: Generate response in English
    response_en = generate_response(user_input_en)
    # Step 3: Translate back to Indic
    final_response = translate(response_en, source_lang="eng_Latn", target_lang=src_lang)
    return final_response

In [24]:
tel_question = "చక్కెర వ్యాధి ఉన్న వ్యక్తి ఎలా జాగ్రత్తలు తీసుకోవాలి?"
print("Answer:", rural_health_bot(tel_question))


Answer: మధుమేహం మందుల వాడకం గ్యాస్ట్రోయింటెస్టినల్ సమస్యలకు కారణమవుతుంది. కొన్ని మందులు ప్రసవ, వాపు, మరియు / లేదా కడుపు నొప్పికి కారణమవుతాయి. మధుమేహం రోగులు కొన్ని మందులను తీసుకోవడం నివారించాలి మరియు ఏదైనా మందులను తీసుకోవడానికి ముందు ఆరోగ్య సంరక్షణ ప్రదాతతో సంప్రదించాలి. గర్భధారణ సమయంలో కొర్టికోస్టెరాయిడ్లు వంటి కొన్ని మందులు తీసుకోకూడదు.


In [25]:
def rural_health_bot(user_input_indic, src_lang="tel_Telu"):
    user_input_en = translate(user_input_indic, source_lang=src_lang, target_lang="eng_Latn")
    response_en = generate_response(user_input_en)
    final_response = translate(response_en, source_lang="eng_Latn", target_lang=src_lang)
    return final_response, user_input_en  # returning both

# Then call it like this:
answer, translated = rural_health_bot(tel_question)
print("EN Input:", translated)
print("Answer:", answer)


EN Input: What precautions should a person with diabetes take?
Answer: మధుమేహం మీ శరీరంలోని రక్త నాళాలను ప్రభావితం చేస్తుంది, ఇది వాటికి వాపు కలిగించవచ్చు. వాపు రక్త నాళాల గోడలకు నష్టం కలిగించవచ్చు, ఇది గుండెపోటు, స్ట్రోక్ మరియు ఆంగినా ప్రమాదాన్ని పెంచుతుంది. వేయించిన ఆహారాలు, ఫాస్ట్ ఫుడ్ మరియు ప్రాసెస్ చేసిన ఆహారాలు వంటి అధిక కొవ్వు కలిగిన ఆహారాలను నివారించడం మరియు ఎక్కువ పండ్లు మరియు కూరగాయలు తినడం రక్తంలో కొలెస్ట్రాల్ స్థాయిలను తగ్గించడానికి సహాయపడుతుంది. క్రమం తప్పకుండా శారీరక శ్రమ కూడా రక్తంలో కొలెస్ట్రాల్ స్థాయిలను తగ్గించడానికి సహాయపడుతుంది.
